## folium

folium是一個建立在Python系統之上的js庫，可以很輕鬆地將在Python中操作的資料視覺化為互動式的單張地圖，且將緊密地將資料與地圖聯絡在一起，可自定義箭頭，網格等HTML格式的地圖示記。該庫還附有一些內建的地形資料。


### - 安裝

    方法一：
    
        pip install folium 
    
    方法二： 
    
        conda install folium 
    
    方法三： 
    
        下載安裝(https://pypi.python.org/pypi/folium/0.5.0)
        

### - 使用

In [ ]:
import folium 

In [ ]:
location = [25, 121.50] # 台灣的經緯度

folium.Map(location=location) 

### 地圖大小、與頂端距離

In [ ]:
folium.Map(location=location, 
           width='50%', 
           height='50%', 
           top='20%'
          ) 

### 地圖樣式類型設定
- OpenStreetMap
- Stamen Terrain
- Stamen Toner
- Stamen Watercolor
- CartoDB positron
- CartoDB dark_matter
- Mapbox Bright
- Mapbox Control Room

In [ ]:
folium.Map(location=location, 
           width='100%', 
           height='50%', 
           tiles = 'CartoDB positron'
          ) 

### 地圖距離縮放

範圍為0~18

In [ ]:
folium.Map(location=location, zoom_start = 5) 

### 添加標記

In [ ]:
m = folium.Map(location=[25.1, 121.5],
                   zoom_start=11,
                   tiles='OpenStreetMap')

folium.Marker([25.033, 121.564099], popup='Taipei 101').add_to(m)
folium.CircleMarker([25.033, 121.564099]).add_to(m)
m

### 分級著色圖

In [ ]:
# get json links
from urllib.request import Request, urlopen, quote
import json
import pandas as pd
import ssl
ssl._create_default_https_context = ssl._create_unverified_context


link = 'http://www.geologycloud.tw/data/zh-tw/liquefaction?area={}&classify={}&all=true'
city = [quote('臺北')] * 3
trend = [quote(t) for t in ['低潛勢', '中潛勢', '高潛勢']]
links = [link.format(x, y) for x, y in zip(city, trend)]
links

In [ ]:
def get_data(url):
    u = urlopen(url)
    dat = json.dumps(json.loads(u.read().decode('utf-8')))
    return dat

low = get_data(links[0])
median = get_data(links[1])
high = get_data(links[2])

In [ ]:
import folium
m = folium.Map(location=[25.05, 121.5], zoom_start=12, tiles='Stamen Terrain')
m.choropleth(low, fill_color='green')
m.choropleth(median, fill_color='yellow')
m.choropleth(high, fill_color='red')
m

# Try: all data on map

### 讀取shapefile進行標記

In [ ]:
# pip install pyshp
import shapefile

In [ ]:
reader = shapefile.Reader('data/空氣品質監測站/空氣品質監測站位置圖_121_10704.shp')
reader.fields

In [ ]:
fields = reader.fields[1:]
field_names = [field[0] for field in fields]
field_names

In [ ]:
buffer = []
for sr in reader.shapeRecords():
    records=sr.record
    for x in [0,2,3,4,5,8]:
        records[x] = records[x].decode('big5')
    atr = dict(zip(field_names, records))
    geom = sr.shape.__geo_interface__
    geom['coordinates'] = tuple([float(records[6]), float(records[7])])
    # geom = sr.shape.__geo_interface__ ## (189901.75652849016, 2646862.0741260033)
    buffer.append(dict(type='Feature', geometry=geom, properties = atr))

In [ ]:
from json import dumps
geojson = open("data/空氣品質監測站_10704.json", "w")
geojson.write(dumps({"type": "FeatureCollection", "features": buffer}, indent=2) + "\n")
geojson.close()

In [ ]:
station_info = json.load(open('data/空氣品質監測站_10704.json'))
stations=json.dumps(station_info)

In [ ]:
m = folium.Map(location=[24.05, 121.5], zoom_start=7,tiles='Stamen Watercolor')
folium.GeoJson(stations).add_to(m)

In [ ]:
m

In [ ]:
import geopandas as gpd

In [ ]:
geo_df = gpd.read_file('data/空氣品質監測站_10704.json')
geo_df.head()

In [ ]:
geo_df.plot()

In [ ]:
m = folium.Map(location=[24.05, 121.5], zoom_start=7,tiles='Stamen Watercolor')
station_info = json.load(open('data/空氣品質監測站_10704.json'))
stations=json.dumps(station_info)

style = lambda x :{'fillColor':'orange'}

lon = geo_df.TWD97Lon.tolist()
lat = geo_df.TWD97Lat.tolist()

site_name = geo_df.SiteName.tolist()
address = geo_df.SiteAddres.tolist()
site_type = geo_df.SiteType.tolist()

for i in range(len(geo_df)):
    
    popup = '站名 : {}<br>地址 : {}<br>測站類型 : {}'
    popup = popup.format(site_name[i], address[i], site_type[i])
    
    
    m.add_child(
        folium.CircleMarker(
            location=[lat[i], lon[i]],
            color='green', 
            radius=10, 
            popup=popup,
            fill=True,
            fill_opacity=0.5
        ))

m